<a href="https://colab.research.google.com/github/Toandinh1/DeepIM/blob/master/10_1_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from scipy.special import binom
from keras.layers.normalization import BatchNormalization
from keras.layers import Lambda, Reshape, Dense, Input, Activation

#from keras import backend as K




N = 4 # number of sub-carriers
K = 1 # number of active sub-carriers
M = 4 # M-ary modulation order

SNRdb = 7 # Training SNR

traing_epochs = 200
l_rate = 0.001
total_batch = 20 # number of batches per epoch
batch_size = 200


n_output_1 = 16
n_output_2 = 32
n_input_1 = N
n_input_2 = 2*N


m = int(np.log2(M))
c = int(np.log2(binom(N,K)))
q = K*m + c # number of bits per OFDM-IM symbol
Q= 2**q
n_output = c

c1 = 4
c2 = 1

SNR = 10**(SNRdb/10)
sigma = np.sqrt(1/SNR)

display_step = 5
qam_factor = (2/3)*(M-1)


a = 1/np.sqrt(2)

# M-ary modulations
if M==4:
    QAM = np.array([1+0j, 0+1j, -0-1j, -1+0j], dtype=complex) # gray mapping
elif M==8:
    QAM = np.array([1, a+a*1j, -a+a*1j, 1j, a-a*1j, -1j, -1, -a-a*1j], dtype=complex) # 8PSK, not 8QAM indeed
    qam_factor = 1
elif M==16:
    QAM = np.array([-3+3j, -3+1j, -3-3j, -3-1j, 
                    -1+3j, -1+1j, -1-3j, -1-1j, 
                    3+3j, 3+1j, 3-3j, 3-1j, 
                    1+3j, 1+1j, 1-3j, 1-1j], dtype=complex)
else:
    QAM = np.array([1,-1], dtype=complex) #BPSK
    qam_factor = 1


# index patterns for N=4 and K=1,2,3 only
if K==1:
    idx = np.array([[0],[1],[2],[3]])
elif K==2:
    idx = np.array([[0,1],[2,3],[0,2],[1,3]]) 
else:
    idx = np.array([[0,1,2],[1,2,3],[0,2,3],[0,1,3]]) 
def SC_IM_NO_train(bit1,bit2, SNRdb):
        #user1
    bit_id1 = bit1[0:c:1]
    id_de1 = bit_id1.dot(2**np.arange(bit_id1.size)[::-1])
    bit_sy1 = bit1[c:q:1]   
    bit_K1 = bit_sy1.reshape(-1,m)
    sy_de1 = np.zeros((K,), dtype=int)
    sym1 = np.zeros((K,), dtype=complex)
    one_hot_bit1 = np.zeros((16,),dtype=int)
    for i in range(K):
        bit_sy_i1 = bit_K1[i,:]
        sy_de1[i] = bit_sy_i1.dot(2**np.arange(bit_sy_i1.size)[::-1])
        sym1[i] = QAM[sy_de1[i]]

    tx_sym1 = np.zeros((N,), dtype=complex)
    tx_sym1[idx[id_de1,:]] = sym1
    tx_sym1 = tx_sym1*np.sqrt(c1)
    one_hot1 = 0
    for i in range(M):
      if bit1[i] == 1:
        one_hot1 = one_hot1 + 2**i
      else:
        one_hot1 = one_hot1 + 0
    one_hot_bit1[one_hot1] = 1
  #user2
    bit_id2 = bit2[0:c:1]
    id_de2 = bit_id2.dot(2**np.arange(bit_id2.size)[::-1])
    bit_sy2 = bit2[c:q:1]   
    bit_K2 = bit_sy2.reshape(-1,m)
    sy_de2 = np.zeros((K,), dtype=int)
    sym2 = np.zeros((K,), dtype=complex)
    one_hot_bit2  = np.zeros((16,),dtype=int)
    for i in range(K):
        bit_sy_i2 = bit_K2[i,:]
        sy_de2[i] = bit_sy_i2.dot(2**np.arange(bit_sy_i2.size)[::-1])
        sym2[i] = QAM[sy_de2[i]]

    tx_sym2 = np.zeros((N,), dtype=complex)
    tx_sym2[idx[id_de2,:]] = sym2
    tx_sym2 = tx_sym2*np.sqrt(c2)
    one_hot2 = 0
    for i in range(M):
      if bit2[i] == 1:
        one_hot2 = one_hot2 + 2**i
      else:
        one_hot2 = one_hot2 + 0
    one_hot_bit2[one_hot2] = 1

    #transmision
    SNR = 10**(SNRdb/10)
    sigma = np.sqrt(1/SNR)
    noise = sigma*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    #noise = np.random.normal(0, 1, tx_sym1.shape)
    #H1 = 1
    #H2 = 1
    H1 = 4*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H2 = np.sqrt(1/2)*(np.random.randn(*tx_sym2.shape)+1j*np.random.randn(*tx_sym2.shape))
    y = H1*tx_sym1 + H2*tx_sym2 + noise
     
    y_bar = y/ H1
    y_con = np.concatenate((np.real(y_bar),np.imag(y_bar)))
    y_m = np.absolute(y_bar)
    Y =np.concatenate((y_con,y_m))
    
    return Y,y_con
    
  
    

def SC_IM_NO_test(bit1, bit2, SNRdb):
        #user1
    bit_id1 = bit1[0:c:1]
    id_de1 = bit_id1.dot(2**np.arange(bit_id1.size)[::-1])
    bit_sy1 = bit1[c:q:1]   
    bit_K1 = bit_sy1.reshape(-1,m)
    sy_de1 = np.zeros((K,), dtype=int)
    sym1 = np.zeros((K,), dtype=complex)
    one_hot_bit1 = np.zeros((16,),dtype=int)
    for i in range(K):
        bit_sy_i1 = bit_K1[i,:]
        sy_de1[i] = bit_sy_i1.dot(2**np.arange(bit_sy_i1.size)[::-1])
        sym1[i] = QAM[sy_de1[i]]

    tx_sym1 = np.zeros((N,), dtype=complex)
    tx_sym1[idx[id_de1,:]] = sym1
    tx_sym1 = tx_sym1*np.sqrt(c1)
    one_hot1 = 0
    for i in range(M):
      if bit1[i] == 1:
        one_hot1 = one_hot1 + 2**i
      else:
        one_hot1 = one_hot1 + 0
    one_hot_bit1[one_hot1] = 1
  #user2
    bit_id2 = bit2[0:c:1]
    id_de2 = bit_id2.dot(2**np.arange(bit_id2.size)[::-1])
    bit_sy2 = bit2[c:q:1]   
    bit_K2 = bit_sy2.reshape(-1,m)
    sy_de2 = np.zeros((K,), dtype=int)
    sym2 = np.zeros((K,), dtype=complex)
    one_hot_bit2 = np.zeros((16,),dtype=int)
    for i in range(K):
        bit_sy_i2 = bit_K2[i,:]
        sy_de2[i] = bit_sy_i2.dot(2**np.arange(bit_sy_i2.size)[::-1])
        sym2[i] = QAM[sy_de2[i]]

    tx_sym2 = np.zeros((N,), dtype=complex)
    tx_sym2[idx[id_de2,:]] = sym2
    tx_sym2 = tx_sym2*np.sqrt(c2)
    one_hot2 = 0
    for i in range(M):
      if bit2[i] == 1:
        one_hot2 = one_hot2 + 2**i
      else:
        one_hot2 = one_hot2 + 0
    one_hot_bit2[one_hot2] = 1

    #transmision
    SNR = 10**(SNRdb/10)
    sigma = np.sqrt(1/SNR)
    noise = sigma*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    #noise = np.random.normal(0, 1, tx_sym1.shape)
    #H1 = 1
    #H2 = 1
    H1 = 4*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H2 = np.sqrt(1/2)*(np.random.randn(*tx_sym2.shape)+1j*np.random.randn(*tx_sym2.shape))
    y = H1*tx_sym1 + H2*tx_sym2 + noise
     
    y_bar = y/ H1
    y_con = np.concatenate((np.real(y_bar),np.imag(y_bar)))
    y_m = np.absolute(y_bar)
    Y =np.concatenate((y_con,y_m))
    
    return Y,y_con
#model     
ini = 'glorot_uniform'
init=tf.global_variables_initializer()
X = tf.placeholder("float", [None, 12])
Y = tf.placeholder("float", [None, 16])
initializer = tf.contrib.layers.xavier_initializer()

def encoderA1(x1):
     weights = {                    
        'encoder_h1': tf.Variable(initializer([12, 16])),
        'encoder_h2': tf.Variable(initializer([12, 16])), 
        'encoder_h3': tf.Variable(initializer([12, 16])),
          
     }
     biases = {            
        'encoder_b1': tf.Variable(initializer([16])),
        'encoder_b2': tf.Variable(initializer([16])),
        'encoder_b3': tf.Variable(initializer([16])),          
     }
    #layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    #layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    #layer_fisrt = Dense(12, activation='linear', init=ini)(x)
    #layer_batch = BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(layer_fisrt)
     layer_1 = Dense(12, activation=tf.nn.relu, init=ini)(x)
     layer_batch = BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(layer_1)
     Q= tf.add(tf.matmul(layer_batch, weights['encoder_h1']), biases['encoder_b1'])
     #K = Dense(16, activation='linear', init=ini)(layer_batch)
     #V = Dense(16, activation='linear', init=ini)(layer_batch)\
     #E = tf.matmul(Q,K)
     K= tf.add(tf.matmul(layer_batch, weights['encoder_h2']), biases['encoder_b2'])
     V= tf.add(tf.matmul(layer_batch, weights['encoder_h3']), biases['encoder_b3'])
     E= tf.matmul(Q, K)
     layer_3 = Dense(12, activation=tf.nn.sigmoid, init=ini)(E)
     A1 = tf.matmul(layer_3,V)
     return A1
def encoderA2(x2):
     weights = {                    
        'encoder_h1': tf.Variable(initializer([12, 16])),
        'encoder_h2': tf.Variable(initializer([12, 16])), 
        'encoder_h3': tf.Variable(initializer([12, 16])),
          
     }
     biases = {            
        'encoder_b1': tf.Variable(initializer([16])),
        'encoder_b2': tf.Variable(initializer([16])),
        'encoder_b3': tf.Variable(initializer([16])),          
     }
    #layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    #layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    #layer_fisrt = Dense(12, activation='linear', init=ini)(x)
    #layer_batch = BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(layer_fisrt)
     layer_1 = Dense(12, activation=tf.nn.relu, init=ini)(x)
     layer_batch = BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(layer_1)
     Q= tf.add(tf.matmul(layer_batch, weights['encoder_h1']), biases['encoder_b1'])
     #K = Dense(16, activation='linear', init=ini)(layer_batch)
     #V = Dense(16, activation='linear', init=ini)(layer_batch)\
     #E = tf.matmul(Q,K)
     K= tf.add(tf.matmul(layer_batch, weights['encoder_h2']), biases['encoder_b2'])
     V= tf.add(tf.matmul(layer_batch, weights['encoder_h3']), biases['encoder_b3'])
     E= tf.matmul(Q, K)
     layer_3 = Dense(12, activation=tf.nn.sigmoid, init=ini)(E)
     A2 = tf.matmul(layer_3,V)
     return A2
def encoder(x3,x4):
    layer_1= tf.concat(x3,x4,axis=-1)
    A = Dense(12, activation='linear', init=ini)(layer_1)
    normA= BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(A)
    P = Dense(12, activation=tf.nn.gelu, init=ini)(normA)
    O = Dense(12, activation='linear', init=ini)(P)
    return O
#y_pred = encoder(X)
#y_true = Y

#cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#learning_rate = tf.placeholder(tf.float32, shape=[])
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#init = tf.global_variables_initializer()
#def detobit(A):
  #x= np.zeros((4,))
  #y= np.zeros((4,))
  #x[0] = A//2
  #y[0] = A%2
  #x[1] = x[0]//2
  #y[1] = x[0]%2
  #x[2] = x[1]//2
  #y[2] = x[1]%2
  #x[3] = x[2]//2
  #y[3] = x[2]%2
  #bit_est =np.array([y[0],y[1],y[2],y[3]],dtype= int)
  #return bit_est

def frange(x,y,jump):
    while x < y:
        yield x
        x +=jump

EbNodB_range = list(frange(0,50,5))
BER1 = [None]*len(EbNodB_range)


with tf.Session() as sess:
#Training
    sess.run(init)
    for epoch in range(traing_epochs):
        avg_cost = 0
        for index_m in range(total_batch):
            input_samples = []
            input_samples1 = []
            input_labels = []
            
            for index_k in range(0, batch_size):
                bits1 = np.random.binomial(n=1,p=0.5,size=(q,))
                bits2 = np.random.binomial(n=1,p=0.5,size=(q,))
                signaloutput,y_con,one_hot_bit1,one_hot_bit2 = SC_IM_NO_train(bits1,bits2,10)
                input_labels.append(one_hot_bit1)
                input_samples.append(signaloutput)
                
               

            batch_x = np.asarray(input_samples)
            batch_xx = np.asarray(input_samples1)
            batch_y = np.asarray(input_labels)
           
            
            

            _,cs = sess.run([optimizer,cost], feed_dict={X:batch_x,
                                                        Y:batch_y,
                                                        learning_rate:l_rate})
            avg_cost += cs / total_batch
        if epoch % display_step == 0:
            print("Epoch:",'%04d' % (epoch+1), "cost=", \
               "{:.9f}".format(avg_cost))
#==========Testing=============
    for n in range(0,len(EbNodB_range)):
      input_samples_test = []
      input_samples1_test = []
      input_labels_test = []
      
      test_number = 100000
      if n>3:
        test_number = 100000
      for i in range(0, test_number):
        bits1 = np.random.binomial(n=1, p=0.5, size=(q, )) 
        bits2 = np.random.binomial(n=1, p=0.5, size=(q, ))
        signaloutput,y_con,one_hot_bit1,one_hot_bit2 = SC_IM_NO_train(bits1,bits2,EbNodB_range[n])
        input_labels_test.append(bits1)
        input_samples_test.append(signaloutput)
        
  
  
      batch_1 = np.asarray(input_samples_test)
      batch_2 = np.asarray(input_labels_test)
      one_hot_bit1_est= sess.run(y_pred,feed_dict={X:batch_1})
      bit_error = 0
      for i in range(0, test_number):
        ind_est =np.argmax(one_hot_bit1_est[i,])
        bit_est = detobit(ind_est)
        bit_error =bit_error+sum(bit_est!=batch_2[i,])
      BER1[n] = bit_error/(test_number*q)
    print("SNR=", EbNodB_range[n], "BER:", BER1[n])
    #ML1=[0.133100000000000,	0.0551562500000000,	0.0170187500000000,	0.00188125000000000,	5.62500000000000e-05,	0,	0]
    import matplotlib.pyplot as plt
    plt.plot(EbNodB_range, BER1, 'bo-',label='DL')
    #plt.plot(EbNodB_range, ML1, 'ro-',label='ML')
    #plt.plot(list(EbNodB_range), ber_theory, 'ro-',label='BPSK BER')
    plt.yscale('log')
    plt.xlabel('SNR Range')
    plt.ylabel('BER')
    plt.grid()
    plt.legend(loc='upper right',ncol = 1)
    plt.savefig('DL_Detection_IM_BER_matplotlib')
    plt.show()


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct  3 14:22:45 2022

@author: GIT3HC
"""

import numpy as np
import tensorflow as tf
from scipy.special import binom
from keras.layers.normalization import BatchNormalization
from keras.layers import Lambda, Reshape, Dense, Input, Activation
import torch
import torchvision
#from keras import backend as K




N = 4 # number of sub-carriers
K = 1 # number of active sub-carriers
M = 4 # M-ary modulation order

SNRdb = 7 # Training SNR

traing_epochs = 200
l_rate = 0.001
total_batch = 20 # number of batches per epoch
batch_size = 200


n_output_1 = 16
n_output_2 = 32
n_input_1 = N
n_input_2 = 2*N


m = int(np.log2(M))
c = int(np.log2(binom(N,K)))
q = K*m + c # number of bits per OFDM-IM symbol
Q= 2**q
n_output = c

c1 = 4
c2 = 1

SNR = 10**(SNRdb/10)
sigma = np.sqrt(1/SNR)

display_step = 5
qam_factor = (2/3)*(M-1)


a = 1/np.sqrt(2)

# M-ary modulations
if M==4:
    QAM = np.array([1+0j, 0+1j, -0-1j, -1+0j], dtype=complex) # gray mapping
elif M==8:
    QAM = np.array([1, a+a*1j, -a+a*1j, 1j, a-a*1j, -1j, -1, -a-a*1j], dtype=complex) # 8PSK, not 8QAM indeed
    qam_factor = 1
elif M==16:
    QAM = np.array([-3+3j, -3+1j, -3-3j, -3-1j, 
                    -1+3j, -1+1j, -1-3j, -1-1j, 
                    3+3j, 3+1j, 3-3j, 3-1j, 
                    1+3j, 1+1j, 1-3j, 1-1j], dtype=complex)
else:
    QAM = np.array([1,-1], dtype=complex) #BPSK
    qam_factor = 1


# index patterns for N=4 and K=1,2,3 only
if K==1:
    idx = np.array([[0],[1],[2],[3]])
elif K==2:
    idx = np.array([[0,1],[2,3],[0,2],[1,3]]) 
else:
    idx = np.array([[0,1,2],[1,2,3],[0,2,3],[0,1,3]]) 
def SC_IM_NO_train(bit1,bit2, SNRdb):
        #user1
    bit_id1 = bit1[0:c:1]
    id_de1 = bit_id1.dot(2**np.arange(bit_id1.size)[::-1])
    bit_sy1 = bit1[c:q:1]   
    bit_K1 = bit_sy1.reshape(-1,m)
    sy_de1 = np.zeros((K,), dtype=int)
    sym1 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i1 = bit_K1[i,:]
        sy_de1[i] = bit_sy_i1.dot(2**np.arange(bit_sy_i1.size)[::-1])
        sym1[i] = QAM[sy_de1[i]]

    tx_sym1 = np.zeros((N,), dtype=complex)
    tx_sym1[idx[id_de1,:]] = sym1
    tx_sym1 = tx_sym1*np.sqrt(c1)
    one_hot1 = np.zeros((N,M+1), dtype=int)
    for i in range(M):
       if tx_sym1[i] == 0+0j:
        one_hot1[i,M] = 1
       else:
        one_hot1[i,M] = 0
        one_hot1[i,id_de1] = 1
  
  #user2
    bit_id2 = bit2[0:c:1]
    id_de2 = bit_id2.dot(2**np.arange(bit_id2.size)[::-1])
    bit_sy2 = bit2[c:q:1]   
    bit_K2 = bit_sy2.reshape(-1,m)
    sy_de2 = np.zeros((K,), dtype=int)
    sym2 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i2 = bit_K2[i,:]
        sy_de2[i] = bit_sy_i2.dot(2**np.arange(bit_sy_i2.size)[::-1])
        sym2[i] = QAM[sy_de2[i]]

    tx_sym2 = np.zeros((N,), dtype=complex)
    tx_sym2[idx[id_de2,:]] = sym2
    tx_sym2 = tx_sym2*np.sqrt(c2)
    one_hot2 = np.zeros((N,M+1), dtype=int)
    for i in range(M):
       if tx_sym2[i] == 0+0j:
        one_hot2[i,M] = 1
       else:
        one_hot2[i,M] = 0
        one_hot2[i,id_de2] = 1

    #transmision
    SNR = 10**(SNRdb/10)
    sigma = np.sqrt(1/SNR)
    noise = sigma*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    #noise = np.random.normal(0, 1, tx_sym1.shape)
    #H1 = 1
    #H2 = 1
    H1 = 4*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H2 = np.sqrt(1/2)*(np.random.randn(*tx_sym2.shape)+1j*np.random.randn(*tx_sym2.shape))
    y = H1*tx_sym1 + H2*tx_sym2 + noise
     
    y_bar = y/ H1
    #y_con = np.concatenate((np.real(y_bar),np.imag(y_bar)),axis=-1)
    y_con = np.stack((np.real(y_bar),np.imag(y_)), axis=1)
    y_m = np.absolute(y_bar)
    Y =np.stack((y_con,y_m), axis=1)
    
    return Y,one_hot1,one_hot2
    
  
    

def SC_IM_NO_test(bit1, bit2, SNRdb):
            #user1
    bit_id1 = bit1[0:c:1]
    id_de1 = bit_id1.dot(2**np.arange(bit_id1.size)[::-1])
    bit_sy1 = bit1[c:q:1]   
    bit_K1 = bit_sy1.reshape(-1,m)
    sy_de1 = np.zeros((K,), dtype=int)
    sym1 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i1 = bit_K1[i,:]
        sy_de1[i] = bit_sy_i1.dot(2**np.arange(bit_sy_i1.size)[::-1])
        sym1[i] = QAM[sy_de1[i]]

    tx_sym1 = np.zeros((N,), dtype=complex)
    tx_sym1[idx[id_de1,:]] = sym1
    tx_sym1 = tx_sym1*np.sqrt(c1)
    one_hot1 = np.zeros((N,M+1), dtype=int)
    for i in range(M):
       if tx_sym1[i] == 0+0j:
        one_hot1[i,M] = 1
       else:
        one_hot1[i,M] = 0
        one_hot1[i,id_de1] = 1
  
  #user2
    bit_id2 = bit2[0:c:1]
    id_de2 = bit_id2.dot(2**np.arange(bit_id2.size)[::-1])
    bit_sy2 = bit2[c:q:1]   
    bit_K2 = bit_sy2.reshape(-1,m)
    sy_de2 = np.zeros((K,), dtype=int)
    sym2 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i2 = bit_K2[i,:]
        sy_de2[i] = bit_sy_i2.dot(2**np.arange(bit_sy_i2.size)[::-1])
        sym2[i] = QAM[sy_de2[i]]

    tx_sym2 = np.zeros((N,), dtype=complex)
    tx_sym2[idx[id_de2,:]] = sym2
    tx_sym2 = tx_sym2*np.sqrt(c2)
    one_hot2 = np.zeros((N,M+1), dtype=int)
    for i in range(M):
       if tx_sym2[i] == 0+0j:
        one_hot2[i,M] = 1
       else:
        one_hot2[i,M] = 0
        one_hot2[i,id_de2] = 1

    #transmision
    SNR = 10**(SNRdb/10)
    sigma = np.sqrt(1/SNR)
    noise = sigma*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    #noise = np.random.normal(0, 1, tx_sym1.shape)
    #H1 = 1
    #H2 = 1
    H1 = 4*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H2 = np.sqrt(1/2)*(np.random.randn(*tx_sym2.shape)+1j*np.random.randn(*tx_sym2.shape))
    y = H1*tx_sym1 + H2*tx_sym2 + noise
     
    y_bar = y/ H1
    #y_con = np.concatenate((np.real(y_bar),np.imag(y_bar)),axis=-1)
    y_con = np.stack((np.real(y_bar),np.imag(y_)), axis=1)
    y_m = np.absolute(y_bar)
    Y =np.stack((y_con,y_m), axis=1)
    
    return Y,one_hot1,one_hot2
#model     
ini = 'glorot_uniform'
init=tf.global_variables_initializer()
X = tf.placeholder("float", [4,3])
Y = tf.placeholder("float", [4,5])
x3 = tf.placeholder("float", [4,3])
x4 = tf.placeholder("float", [4,3])
initializer = tf.contrib.layers.xavier_initializer()


def encoderA1(x1):
     layer_1 = Dense(5, activation=tf.nn.relu, init=ini)(x1)
     layer_batch = BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(layer_1)
     Q = Dense(3, activation='linear', init=ini)(layer_batch)    
     K = Dense(3, activation='linear', init=ini)(layer_batch)
     V = Dense(3, activation='linear', init=ini)(layer_batch)
     E = tf.matmul(Q,tf.transpose(K))
     layer_3 = Dense(4, activation=tf.nn.sigmoid, init=ini)(E)
     A1 = tf.matmul(layer_3,V)
     return A1
def encoderA2(x2):
     layer_1 = Dense(5, activation=tf.nn.relu, init=ini)(x2)
     layer_batch = BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(layer_1)
     Q = Dense(3, activation='linear', init=ini)(layer_batch)    
     K = Dense(3, activation='linear', init=ini)(layer_batch)
     V = Dense(3, activation='linear', init=ini)(layer_batch)
     E = tf.matmul(Q,tf.math.transpose(K))
     layer_3 = Dense(4, activation=tf.nn.sigmoid, init=ini)(E)
     A2 = tf.matmul(layer_3,V)
     return A2
def encoder(x3,x4):
    A = tf.concat((x3,x4), axis=1)
    normA= BatchNormalization(momentum=0.99, epsilon=0.00001, center=True,scale=True)(A)
    P = Dense(4, activation=tf.nn.relu, init=ini)(normA)
    O = Dense(5, activation='linear', init=ini)(P)
    return O
X1 = encoderA1(X)
X2 = encoderA2(X)
y_pred = encoder(X1,X2)
y_true = Y

cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
learning_rate = tf.placeholder(tf.float32, shape=[])
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.global_variables_initializer()

